# Charpter4: C Performance with Cython 

## Compiling Cython entensions

### Cythonmagic

In [1]:
%load_ext Cython

In [3]:
%%cython
def hello_snippet():
    print("\nHello, Cython!")

hello_snippet()


Hello, Cython!


## Static types

### Declaring variables

In [11]:
%%cython
cpdef int max_cython(int a, int b):
    return a if a > b else b

In [26]:
max_cython(1, 2)

2

### Declaring classes

In [22]:
%%cython
cdef class Point:
    cdef public double x
    cdef public double y
    def __init__(self, double x, double y):
        self.x = x
        self.y = y

Content of stderr:
ld: warning: duplicate -rpath '/opt/anaconda3/lib' ignored

In [23]:
a = Point(0, 0)
a.x

0.0

## Working with arrays

### C arrays and pointers

In [107]:
%%cython
cdef double a
from libc.stdio cimport printf
printf("\n%p", &a)

In [109]:
%%cython --force
from libc.stdio cimport printf
cdef double a
cdef double *a_pointer
a_pointer = &a

a = 3.0
print("\n")
print(a_pointer[0])

Content of stderr:
ld: warning: duplicate -rpath '/opt/anaconda3/lib' ignored

3.0


In [117]:
%%cython
cdef double arr1[10]
cdef double arr2[5][2]
arr1[0]
arr2[1]

Content of stderr:
ld: warning: duplicate -rpath '/opt/anaconda3/lib' ignored

In [119]:
%%cython
from libc.stdio cimport printf
cdef double arr[10]
printf("%p\n", arr)
printf("%p\n", &arr[0])

Content of stderr:
ld: warning: duplicate -rpath '/opt/anaconda3/lib' ignored

### Working with NumPy arrays

In [139]:
%%cython
import numpy as np
def numpy_bench_py():
    py_arr = np.random.rand(1000)
    cdef int i
    
    for i in range(1000):
        py_arr[i] += 1

In [141]:
%%cython
import numpy as np
cimport numpy as c_np
def numpy_bench_c():
    cdef c_np.ndarray[double, ndim=1] c_arr
    c_arr = np.random.rand(1000)
    cdef int i
    
    for i in range(1000):
        c_arr[i] += 1

Content of stderr:
In file included from /Users/luowen/.cache/ipython/cython/_cython_magic_f75d19154b7770dbea51c6be91ccb5301be1af4b.c:1255:
In file included from /opt/anaconda3/lib/python3.12/site-packages/numpy/core/include/numpy/arrayobject.h:5:
In file included from /opt/anaconda3/lib/python3.12/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /opt/anaconda3/lib/python3.12/site-packages/numpy/core/include/numpy/ndarraytypes.h:1929:
/opt/anaconda3/lib/python3.12/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
#warning "Using deprecated NumPy API, disable it with " \
 ^
1 warning generated.
ld: warning: duplicate -rpath '/opt/anaconda3/lib' ignored

In [143]:
%timeit numpy_bench_py()
%timeit numpy_bench_c()

110 μs ± 238 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
4.27 μs ± 6.14 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### Working with typed memoryviews

In [164]:
%%cython --force
import numpy as np
cdef int[:] a
cdef double[:, :] b
a_np = np.zeros(10, dtype="int32")
a = a_np
a[2] = 1
print("\n")
print(a_np)
print(*a)

Content of stderr:
ld: warning: duplicate -rpath '/opt/anaconda3/lib' ignored

[0 0 1 0 0 0 0 0 0 0]
0 0 1 0 0 0 0 0 0 0


In [176]:
%%cython --force
import numpy as np
cdef double[:, :] b
cdef double[:] r
b = np.random.rand(10, 3)
r = np.zeros(3, dtype="float64")
b[0, :] = r
print("\n")
print(*b[0])

Content of stderr:
ld: warning: duplicate -rpath '/opt/anaconda3/lib' ignored

0.0 0.0 0.0


## Using a particle simulator in Cython

In [179]:
from simul import benchmark

In [181]:
%timeit benchmark(100, 'cython')
%timeit benchmark(100, 'numpy')

2.62 ms ± 1.03 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
84.7 ms ± 5.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Profiling Cython

In [29]:
%%cython -a
import numpy as np

cdef int max(int a, int b):
    return a if a > b else b

cdef int chebyshev(int x1, int y1, int x2, int y2):
    return max(abs(x1-x2), abs(y1-y2))

def c_benchmark():
    a = np.random.rand(1000, 2)
    b = np.random.rand(1000, 2)

    for x1, y1 in a:
        for x2, y2 in b:
            chebyshev(x1, x2, y1, y2)

Content of stderr:
ld: warning: duplicate -rpath '/opt/anaconda3/lib' ignored

In [23]:
%timeit c_benchmark()

447 ms ± 98.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
def max(a, b):
    return a if a > b else b

def chebyshev_py(x1, y1, x2, y2):
    return max(abs(x1-x2), abs(y1-y2))

def c_benchmark():
    a = np.random.rand(1000, 2)
    b = np.random.rand(1000, 2)

    for x1, y1 in a:
        for x2, y2 in b:
            chebyshev_py(x1, x2, y1, y2)

In [37]:
%timeit c_benchmark()

541 ms ± 5.22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Charpter5: Exploring Compilers

## Getting started with Numba

### Using Numba decorators

In [194]:
import numba as nb

@nb.jit
def sum_sq(a):
    result = 0
    N = len(a)
    for i in range(N):
        result += a[i]
    return result

In [198]:
import numpy as np

x = np.random.rand(1000)

%timeit sum_sq.py_func(x)
%timeit sum_sq(x)

72.7 μs ± 1.03 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
The slowest run took 5.16 times longer than the fastest. This could mean that an intermediate result is being cached.
1.77 μs ± 1.56 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [200]:
@nb.jit((nb.float64[:],))
def sum_sq(a):
    result = 0
    N = len(a)
    for i in range(N):
        result += a[i]
    return result

In [202]:
sum_sq(x.astype('float32'))

TypeError: No matching definition for argument type(s) array(float32, 1d, C)

In [210]:
@nb.jit((["float64(float64[:])","float32(float32[:])"]))
def sum_sq(a):
    result = 0
    N = len(a)
    for i in range(N):
        result += a[i]
    return result

In [212]:
sum_sq(x.astype('float32'))

508.933837890625

### Object mode versus native mode

In [215]:
@nb.jit
def concatenate(strings):
    result = ''
    for s in strings:
        result += s
    return result

In [219]:
concatenate.inspect_types()

In [221]:
concatenate.signatures

[]

In [223]:
concatenate(["123", "abs"])

'123abs'

In [225]:
concatenate.inspect_types()

concatenate (List(unicode_type, True),)
--------------------------------------------------------------------------------
# File: /var/folders/cc/sms4rc0n4599htcqgybq1nsm0000gn/T/ipykernel_58808/3573004402.py
# --- LINE 1 --- 
# label 0
#   strings = arg(0, name=strings)  :: reflected list(unicode_type)<iv=None>

@nb.jit

# --- LINE 2 --- 

def concatenate(strings):

    # --- LINE 3 --- 
    #   result = const(str, )  :: Literal[str]()
    #   result.2 = result  :: unicode_type
    #   del result

    result = ''

    # --- LINE 4 --- 
    #   $10get_iter.2 = getiter(value=strings)  :: iter(reflected list(unicode_type)<iv=None>)
    #   del strings
    #   $phi12.0 = $10get_iter.2  :: iter(reflected list(unicode_type)<iv=None>)
    #   del $10get_iter.2
    #   jump 12
    # label 12
    #   $12for_iter.1 = iternext(value=$phi12.0)  :: pair<unicode_type, bool>
    #   $12for_iter.2 = pair_first(value=$12for_iter.1)  :: unicode_type
    #   $12for_iter.3 = pair_second(value=$12for_iter.

In [227]:
concatenate.signatures

[(List(unicode_type, True),)]

In [229]:
x = ['hello'] * 1000
%timeit concatenate.py_func(x)
%timeit concatenate(x)

39.5 μs ± 4.73 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
553 μs ± 12.2 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Numba and NumPy

#### Universal functions with Numba

In [233]:
import numpy as np

@np.vectorize
def cantor_py(a, b):
    return  int(0.5 * (a + b)*(a + b + 1) + b)

In [235]:
@nb.vectorize
def cantor(a, b):
    return  int(0.5 * (a + b)*(a + b + 1) + b)

In [237]:
x1 = np.random.rand(10000)
x2 = np.random.rand(10000)

In [239]:
%timeit cantor_py(x1, x2)
%timeit cantor(x1, x2)

1.56 ms ± 12.2 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.37 μs ± 36.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


#### Generalized universal functions

In [242]:
@nb.guvectorize(['float64[:], float64[:], float64[:]'], '(n), (n) -> ()')
def euclidean(a, b, out):
    N = a.shape[0]
    out[0] = 0
    for i in range(N):
        out[0] += (a[i] - b[i])**2  

In [244]:
a = np.random.rand(10000, 2)
b = np.random.rand(10000, 2)

In [250]:
(((a-b)**2).sum(axis=1)) - euclidean(a, b)

array([0., 0., 0., ..., 0., 0., 0.])

In [252]:
%timeit (((a-b)**2).sum(axis=1))
%timeit euclidean(a, b)

144 μs ± 39.1 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
12.8 μs ± 855 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### JIT classes

In [259]:
from numba.experimental import jitclass

In [261]:
node_type = nb.deferred_type()

node_spec = [
    ('next', nb.optional(node_type)),
    ('value', nb.int64)
]

@jitclass(node_spec)
class Node:
    def __init__(self, value):
        self.next = None
        self.value = value

node_type.define(Node.class_type.instance_type)


ll_spec = [
    ('head', nb.optional(Node.class_type.instance_type))
]

@jitclass(ll_spec)
class LinkedList:
    def __init__(self):
        self.head = None
    
    def push_front(self, value):
        if self.head is None:
            self.head = Node(value)
        else:
            # We replace the head
            new_head = Node(value)
            new_head.next = self.head
            self.head = new_head
    
    def show(self):
        node = self.head
        while node is not None:
            print(node.value)
            node = node.next
@nb.jit   
def sum_list(lst):
    result = 0
    node = lst.head
    while node is not None:
        result += node.value
        node = node.next
    return result

In [263]:
lst = LinkedList()
[lst.push_front(i) for i in range(10000)]

%timeit sum_list(lst)
%timeit sum_list.py_func(lst)

51.7 μs ± 885 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
1.49 ms ± 62.1 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Limitation in Numba

In [282]:
a = [[0, 1, 2], 
      [3, 4], 
      [5, 6, 7, 8]]

@nb.jit
def sum_sublists(a):
    result = [0]

    for sublist in a:
        result.append(sum(sublist))
    
    return result[1:]

sum_sublists(a)

TypeError: cannot reflect element of reflected container: reflected list(reflected list(int64)<iv=None>)<iv=None>
